In [4]:
!pip install audio2numpy
!brew install ffmpeg

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/cask).
==> Updated Casks
stack

==> Downloading https://homebrew.bintray.com/bottles/dav1d-0.7.0.catalina.bottle
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/329dab5c4986351c7b360bcc827138458f97e430fc77a3c9928c9f18be1c6aee--dav1d-0.7.0.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/freetype-2.10.1.catalina.bo
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/95d06c9193535a90157efef36a78f373d2a1a697235d9a46f93d2d0cafe44643--freetype-2.10.1.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/frei0r-1.7.0.catalina.bottl
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/3cb327077ff8f803c5ae2468f0dbb9db0d0117589ad5fb25a993397752641881--frei0r-1.7.0.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/gmp-6.2.0.catalina.bottle.t
Already downloaded: /Users/elijah/Library/Caches/Homebrew/do

Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/0275c2a9baf97b7d3cc58d0664b3b229dd51e6aadc400f676bcc05682c1ca924--libvpx-1.8.2.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/jpeg-9d.catalina.bottle.tar
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/9ed6b08679bb12bb63055c727e5cdcb7f0813e6a89a66ee35a2b2b83d1c30ac9--jpeg-9d.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/libtiff-4.1.0.catalina.bott
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/91125427e45248202fc877102b1db0b23238c19fd228950012da491e2d75764b--libtiff-4.1.0.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/rtmpdump-2.4%2B20151223_1.c
Already downloaded: /Users/elijah/Library/Caches/Homebrew/downloads/ec1d46b8a83864cb12a9aa6cafcef368a5f8e4a09d7f2baf6756140356c4780d--rtmpdump-2.4+20151223_1.catalina.bottle.tar.gz
==> Downloading https://homebrew.bintray.com/bottles/flac-1.3.3.catali

In [1]:
from audio2numpy import open_audio

music_path = "/Users/elijah/Downloads/dataset/DANCE_R_3/audio.mp3"

signal, sampling_rate = open_audio (music_path)

print (signal, sampling_rate)

NoBackendError: It is likely that ffmpeg is not yet installed. Please refer github repo for instruction. 
    MacOS: brew install ffmpeg.
    Linux: sudo apt-get install ffmpeg
    Windows: Download distribution from ffmpeg website, unzip, add the path of bin (e.g. `C:fmpegin`) to system PATH.

In [3]:
import soundfile as sf

y, sr = sf.read("/Users/elijah/Downloads/dataset/DANCE_R_3/audio.mp3")

ModuleNotFoundError: No module named 'soundfile'

In [ ]:
import torch
import torch.nn as nn
from generate_data import *
import matplotlib.pyplot as plt

#####################
# Set parameters
#####################

# Data params
noise_var = 0
num_datapoints = 100
test_size = 0.2
num_train = int((1-test_size) * num_datapoints)

# Network params
input_size = 20
# If `per_element` is True, then LSTM reads in one timestep at a time.
per_element = True
if per_element:
    lstm_input_size = 1
else:
    lstm_input_size = input_size
# size of hidden layers
h1 = 32
output_dim = 1
num_layers = 2
learning_rate = 1e-3
num_epochs = 500
dtype = torch.float

#####################
# Generate data
#####################
data = ARData(num_datapoints, num_prev=input_size, test_size=test_size, noise_var=noise_var, coeffs=fixed_ar_coefficients[input_size])

# make training and test sets in torch
X_train = torch.from_numpy(data.X_train).type(torch.Tensor)
X_test = torch.from_numpy(data.X_test).type(torch.Tensor)
y_train = torch.from_numpy(data.y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(data.y_test).type(torch.Tensor).view(-1)

X_train = X_train.view([input_size, -1, 1])
X_test = X_test.view([input_size, -1, 1])

#####################
# Build model
#####################

# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

model = LSTM(lstm_input_size, h1, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

#####################
# Plot preds and performance
#####################

plt.plot(y_pred.detach().numpy(), label="Preds")
plt.plot(y_train.detach().numpy(), label="Data")
plt.legend()
plt.show()

plt.plot(hist, label="Training loss")
plt.legend()
plt.show()